In [11]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

class Edge:
    def __init__(self, from_point, to_point):
        self.from_point = from_point
        self.to_point = to_point

class FlowchartShape(ABC):
    def __init__(self):
        self.neighbors = []
        self.edges = []
        self.anchors = []

    @abstractmethod
    def draw(self, canvas, paint):
        pass

    @abstractmethod
    def draw_mat(self, mat, scalar, thickness):
        pass

    @abstractmethod
    def get_center(self):
        pass

    def get_closest_anchor(self, point):
        min_dist = float('inf')
        res = self.anchors[0]
        for anchor in self.anchors:
            dist = self.get_distance(point, anchor)
            if dist < min_dist:
                min_dist = dist
                res = anchor
        return res

    def get_closest_anchor_dist(self, point):
        min_dist = float('inf')
        for anchor in self.anchors:
            dist = self.get_distance(point, anchor)
            if dist < min_dist:
                min_dist = dist
        return min_dist

    @staticmethod
    def get_distance(from_point, to_point):
        return ((from_point.y - to_point.y) ** 2 + (from_point.x - to_point.x) ** 2) ** 0.5

    @staticmethod
    def point_translator(point):
        return (point.x, point.y)

class Rectangle(FlowchartShape):
    def __init__(self, center, width, height):
        super().__init__()
        self.center = center
        self.width = width
        self.height = height
        self.half_width = width // 2
        self.half_height = height // 2
        self.anchors.extend([
            Point(center.x, center.y - self.half_height),
            Point(center.x, center.y + self.half_height),
            Point(center.x - self.half_width, center.y),
            Point(center.x + self.half_width, center.y)
        ])

    def draw(self, canvas, paint):
        canvas.draw_rect(
            self.center.x - self.half_width,
            self.center.y - self.half_height,
            self.center.x + self.half_width,
            self.center.y + self.half_height,
            paint
        )

    def draw_mat(self, mat, scalar, thickness):
        mat.rectangle(
            (self.center.x - self.half_width, self.center.y - self.half_height),
            (self.center.x + self.half_width, self.center.y + self.half_height),
            scalar,
            thickness
        )

    def get_center(self):
        return self.center

class Rhombus(FlowchartShape):
    def __init__(self, center, width, height):
        super().__init__()
        self.center = center
        self.width = width
        self.height = height
        self.half_width = width // 2
        self.half_height = height // 2
        self.anchors.extend([
            Point(center.x, center.y - self.half_height),
            Point(center.x, center.y + self.half_height),
            Point(center.x - self.half_width, center.y),
            Point(center.x + self.half_width, center.y)
        ])

    def draw(self, canvas, paint):
        path = Path()
        path.move_to(self.center.x, self.center.y + self.half_height)  # Top
        path.line_to(self.center.x - self.half_width, self.center.y)    # Left
        path.line_to(self.center.x, self.center.y - self.half_height)  # Bottom
        path.line_to(self.center.x + self.half_width, self.center.y)   # Right
        path.close()
        canvas.draw_path(path, paint)

    def draw_mat(self, mat, scalar, thickness):
        mat.line((self.center.x, self.center.y - self.half_height), (self.center.x - self.half_width, self.center.y), scalar, thickness)
        mat.line((self.center.x - self.half_width, self.center.y), (self.center.x, self.center.y + self.half_height), scalar, thickness)
        mat.line((self.center.x, self.center.y + self.half_height), (self.center.x + self.half_width, self.center.y), scalar, thickness)
        mat.line((self.center.x + self.half_width, self.center.y), (self.center.x, self.center.y - self.half_height), scalar, thickness)

    def get_center(self):
        return self.center

In [21]:
class Graph:
    # def __init__(self):
    #     self.flowchartShapes = []
    #     self.canvas = None
    #     # Initialize shapes
    #     shape1 = Rectangle(Point(300, 200), 200, 100)
    #     shape2 = Rectangle(Point(600, 200), 200, 100)
    #     shape3 = Rectangle(Point(900, 200), 200, 100)
    #     shape4 = Rhombus(Point(600, 350), 200, 100)
    #     shape5 = Rectangle(Point(300, 500), 200, 100)
    #     shape6 = Rectangle(Point(900, 500), 200, 100)
    #     self.flowchartShapes.extend([shape1, shape2, shape3, shape4, shape5, shape6])
    #     shape1.neighbors.append(shape2)
    #     shape3.neighbors.append(shape2)
    #     shape2.neighbors.append(shape4)
    #     shape4.neighbors.extend([shape5, shape6])
    #     shape1.edges.append(Edge(Point(400, 200), Point(500, 200)))
    #     shape3.edges.append(Edge(Point(800, 200), Point(700, 200)))
    #     shape2.edges.append(Edge(Point(600, 250), Point(600, 300)))
    #     shape4.edges.append(Edge(Point(500, 350), Point(300, 450)))
    #     shape4.edges.append(Edge(Point(700, 350), Point(900, 450)))

    def __init__(self, shapes, edges):
        self.flowchartShapes = []
        if not shapes:
            return
        for edge in edges:
            from_point = edge.from_point
            to_point = edge.to_point
            closest_from = self.getClosestFlowchartShape(shapes, from_point)
            closest_to = self.getClosestFlowchartShape(shapes, to_point)
            anchor_from = closest_from.get_closest_anchor(from_point)
            anchor_to = closest_to.get_closest_anchor(to_point)
            closest_from.neighbors.append(closest_to)
            closest_from.edges.append(Edge(anchor_from, anchor_to))
        self.flowchartShapes.extend(shapes)

    def getClosestFlowchartShape(self, shapes, point):
        min_dist = float('inf')
        closest_shape = shapes[0]
        for shape in shapes:
            dist = shape.get_closest_anchor_dist(point)
            if dist < min_dist:
                min_dist = dist
                closest_shape = shape
        return closest_shape

    def draw(self):
        paint = Paint()
        paint.color = Color.WHITE
        self.drawShapes(paint)
        paint.color = Color.BLACK
        paint.stroke_width = 3
        self.drawArrows(paint)

    def draw(self, mat):
        scalar = (0, 255, 0)
        thickness = 2
        tipLength = 0.3
        self.drawShapes(mat, scalar, thickness)
        self.drawArrows(mat, scalar, thickness, tipLength)

    def setCanvas(self, canvas):
        self.canvas = canvas

    def drawShapes(self, mat_or_paint, scalar_or_paint, thickness=None):
        for shape in self.flowchartShapes:
            shape.draw(mat_or_paint, scalar_or_paint, thickness)

    def drawArrows(self, mat_or_paint, scalar_or_paint, thickness=None, tipLength=None):
        indegree = {shape: 0 for shape in self.flowchartShapes}
        for shape in self.flowchartShapes:
            for to_shape in shape.neighbors:
                indegree[to_shape] += 1
        queue = deque([s for s in indegree if indegree[s] == 0])
        while queue:
            from_shape = queue.popleft()
            for i, to_shape in enumerate(from_shape.neighbors):
                edge = from_shape.edges[i]
                self.drawArrow(edge, mat_or_paint, scalar_or_paint, thickness, tipLength)
                indegree[to_shape] -= 1
                if indegree[to_shape] == 0:
                    queue.append(to_shape)

    def drawArrow(self, edge, mat_or_paint, scalar_or_paint, thickness=None, tipLength=None):
        from_point = edge.from_point
        to_point = edge.to_point
        self.drawArrowLine(from_point, to_point, mat_or_paint, scalar_or_paint, thickness, tipLength)

    def drawArrowLine(self, from_point, to_point, mat_or_paint, scalar_or_paint, thickness=None, tipLength=None):
        if isinstance(mat_or_paint, cv2.UMat):
            cv2.arrowedLine(mat_or_paint, (int(from_point.x), int(from_point.y)), (int(to_point.x), int(to_point.y)), scalar_or_paint, thickness, line_type=8, shift=0, tipLength=tipLength)
        else:
            mat_or_paint.drawLine(from_point.x, from_point.y, to_point.x, to_point.y, scalar_or_paint)

In [14]:
class ProgressBar:

    def __init__(self, leftX, leftY, rightX, rightY):
        self.ratio = 0
        self.IN_PROGRESS_COLOR = (0, 0, 255)
        self.OUT_PROGRESS_COLOR = (255, 0, 0)
        self.leftX = leftX
        self.leftY = leftY
        self.rightX = rightX
        self.rightY = rightY

    def draw(self, mat):
        THICK = 5
        self.drawLeft(mat, THICK)
        self.drawRight(mat, THICK)

    def drawLeft(self, mat, thick):
        rightTemp = int(self.leftX + (self.rightX - self.leftX) * self.ratio)
        cv2.rectangle(mat, (int(self.leftX), int(self.leftY)), (int(rightTemp), int(self.rightY)), self.IN_PROGRESS_COLOR, thick)

    def drawRight(self, mat, thick):
        leftTemp = int(self.leftX + (self.rightX - self.leftX) * self.ratio)
        cv2.rectangle(mat, (int(leftTemp), int(self.leftY)), (int(self.rightX), int(self.rightY)), self.OUT_PROGRESS_COLOR, thick)

    def reset(self):
        self.ratio = 0

    def setRatio(self, ratio):
        self.ratio = ratio


In [15]:
import cv2
import numpy as np

# Assuming that the following classes are available or need to be defined
# Edge, FlowchartShape, Point, Rectangle, Rhombus, Graph, ProgressBar

class Helper:
    @staticmethod
    def BuildArrows(r, center, tails):
        edges = []
        for i in range(r):
            e = Edge(Point(tails[i][0], tails[i][1]), Point(center[i][0], center[i][1]))
            edges.append(e)
        return edges

    @staticmethod
    def BuildRectangles(r, anchors, center):
        rect = []
        for i in range(r):
            width = anchors[i][0]
            height = anchors[i][1]
            shape = Rectangle(Point(center[i][0], center[i][1]), width, height)
            rect.append(shape)
        return rect

    @staticmethod
    def BuildDiamonds(r, anchors, center):
        diamonds = []
        for i in range(r):
            width = anchors[i][0]
            height = anchors[i][1]
            shape = Rhombus(Point(center[i][0], center[i][1]), width, height)
            diamonds.append(shape)
        return diamonds

    @staticmethod
    def getGraph(img, progressBar=None):
        Helper.setProgressBarRatio(progressBar, 0)

        height, width = img.shape[:2]

        Helper.setProgressBarRatio(progressBar, 0.2)
        pre = PrePro.prepro(img)
        Helper.setProgressBarRatio(progressBar, 0.5)

        # Extract processed images
        matRectangle = pre[0]
        matDiamond = pre[1]
        matArrow = pre[2]

        print("Height", height)
        print("Width", width)

        # Connected components analysis
        num_labels_rect, labels_rect, stats_rect, centroids_rect = cv2.connectedComponentsWithStats(matRectangle)
        Helper.setProgressBarRatio(progressBar, 0.6)

        num_labels_diamond, labels_diamond, stats_diamond, centroids_diamond = cv2.connectedComponentsWithStats(matDiamond)
        Helper.setProgressBarRatio(progressBar, 0.6)

        num_labels_arrow, labels_arrow, stats_arrow, centroids_arrow = cv2.connectedComponentsWithStats(matArrow)
        Helper.setProgressBarRatio(progressBar, 0.7)

        print("Regions (Rectangles):", num_labels_rect)
        print("Regions (Diamonds):", num_labels_diamond)
        print("Regions (Arrows):", num_labels_arrow)

        r_rect = num_labels_rect - 1
        r_diamond = num_labels_diamond - 1
        r_arrow = num_labels_arrow - 1

        # Find centers
        center_rect = centroids_rect[1:].astype(int)
        center_diamond = centroids_diamond[1:].astype(int)
        center_arrow = centroids_arrow[1:].astype(int)
        Helper.setProgressBarRatio(progressBar, 0.8)

        # Anchors for rectangles and diamonds
        anchors_rect = stats_rect[1:, [2, 3]]  # Width and height
        anchors_diamond = stats_diamond[1:, [2, 3]]
        Helper.setProgressBarRatio(progressBar, 0.9)

        # Tails and heads for arrows
        tails = []
        heads = []
        for i in range(r_arrow):
            x = stats_arrow[i+1, cv2.CC_STAT_LEFT]
            y = stats_arrow[i+1, cv2.CC_STAT_TOP]
            w = stats_arrow[i+1, cv2.CC_STAT_WIDTH]
            h = stats_arrow[i+1, cv2.CC_STAT_HEIGHT]
            center_x, center_y = center_arrow[i]

            box_X = x + 0.5 * w
            box_Y = y + 0.5 * h
            if center_x < box_X:
                if center_y < box_Y:
                    head = [x, y]
                    tail = [x + w, y + h]
                else:
                    head = [x, y + h]
                    tail = [x + w, y]
            else:
                if center_y < box_Y:
                    head = [x + w, y]
                    tail = [x, y + h]
                else:
                    head = [x + w, y + h]
                    tail = [x, y]
            heads.append(head)
            tails.append(tail)
        tails = np.array(tails)
        heads = np.array(heads)

        Helper.setProgressBarRatio(progressBar, 0.9)

        # Build shapes and edges
        rect_shapes = Helper.BuildRectangles(r_rect, anchors_rect, center_rect)
        diamond_shapes = Helper.BuildDiamonds(r_diamond, anchors_diamond, center_diamond)
        shapes = rect_shapes + diamond_shapes

        edges = Helper.BuildArrows(r_arrow, heads, tails)

        # Build graph
        graph = Graph(shapes, edges)
        Helper.setProgressBarRatio(progressBar, 1.0)
        return graph

    @staticmethod
    def setProgressBarRatio(progressBar, ratio):
        if progressBar is None:
            return
        progressBar.setRatio(ratio)


In [22]:
result = Helper.getGraph(fc)

Hello, World!
Grayscale Done!
Binarization Done!
BitWiseNot Done!
Denoise Done!
GetEdge Done!
Rotation Done!
Denoise Done!
Open Done!
Denoise Done!
Diff Done!
Denoise Done!
Dilate Done!
Blob Done!
Erode Done!
aha
aha
Rectangle Diamond Done!
Erode Done!
Erode Done!
Box Done!
Conversion Done!
Height 1041
Width 1600
Regions (Rectangles): 2
Regions (Diamonds): 1
Regions (Arrows): 3


In [25]:
result